In [1]:
!pip install pandas scikit-learn optuna

In [2]:
import pandas as pd
from include.utils import incrementar_dados_aleatorios_csv, categorizar_imc
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import OneHotEncoder, LabelEncoder, StandardScaler
import optuna

**1. Engenharia de Recursos**

Novos recursos derivados dos dados existentes

In [3]:
# leia os dados CSV indicados no site da FIAP
dados = pd.read_csv('../base_dados.csv', encoding='latin-1')

# aqui chamo a função incrementar_dados_aleatorios, para aumentar o volume da variável dados
dados = incrementar_dados_aleatorios_csv(dados)

# Aplicar a função para categorizar IMC e criar uma nova coluna 'Categoria IMC'
dados['Categoria IMC'] = dados['IMC'].apply(categorizar_imc)

**2. Limpeza de Dados**

In [4]:
# Remover linhas com valores NaN
dados = dados.dropna()

# Substituir valores nulos por zero
dados['Filhos'] = dados['Filhos'].fillna(0)
dados['Filhos'] = dados['Filhos'].astype(int)
dados['Idade'] = dados['Idade'].fillna(0)
dados['Idade'] = dados['Idade'].astype(int)
dados['Encargos'] = dados['Encargos'].fillna(0)

dados['Gênero'] = dados['Gênero'].str.replace('nan', 'Não informado')

**3. Pré-processamento**

In [5]:
# Dividir os dados em features (X) e target (y)
X = dados.drop('Encargos', axis=1)  # Substitua 'target_column_name' pelo nome da coluna alvo
y = dados['Encargos']

# Codificar variáveis categóricas usando one-hot encoding
encoder = OneHotEncoder(drop='first')
X_encoded = encoder.fit_transform(X[['Gênero', 'Região']])

# Concatenar features codificadas com as restantes
X_encoded = pd.DataFrame(X_encoded.toarray(), columns=encoder.get_feature_names_out(['Gênero', 'Região']), index=X.index)
X = pd.concat([X.drop(['Gênero', 'Região'], axis=1), X_encoded], axis=1)

# Codificar variável categórica 'IMC' com LabelEncoder
label_encoder = LabelEncoder()
X['Categoria IMC'] = label_encoder.fit_transform(X['Categoria IMC'])

# Substituir valores 'sim' e 'não' por 1 e 0
X['Fumante'] = X['Fumante'].map({'sim': 1, 'não': 0})

# Dividir os dados em conjuntos de treinamento e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [6]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

**4. Treinamento do Modelo**

In [7]:
# Treinar e avaliar os modelos de regressão
models = {
    'Random Forest Regressor': RandomForestRegressor(),
    'Linear Regression': LinearRegression()
}

for name, model in models.items():
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    mse = mean_squared_error(y_test, y_pred)
    print(f'{name} MSE: {round(mse,2)}')
    # Calcular acurácia média usando validação cruzada
    scores = cross_val_score(model, X_train, y_train, cv=5)
    print(f'Acurácia média com validação cruzada para {name}: {round(scores.mean(),2)}')

Random Forest Regressor MSE: 42686568.27
Acurácia média com validação cruzada para Random Forest Regressor: 0.65
Linear Regression MSE: 17734922.84
Acurácia média com validação cruzada para Linear Regression: 0.73


**5. Comparação de Modelos**

In [8]:
# Seleção de modelo com base na menor pontuação de validação cruzada
best_model_name = min(models, key=lambda model: cross_val_score(models[model], X_train, y_train, cv=5).mean())

**6. Seleção de Modelo**

In [9]:
best_model = models[best_model_name]
print(f'Best Model: {best_model_name}')

Best Model: Random Forest Regressor


**7. Otimização de Modelo**

In [10]:
# Otimização de hiperparâmetros com Optuna (exemplo com RandomForestRegressor)
def objective(trial):
    n_estimators = trial.suggest_int('n_estimators', 10, 100)
    max_depth = trial.suggest_int('max_depth', 2, 32)
    model = RandomForestRegressor(n_estimators=n_estimators, max_depth=max_depth)
    score = cross_val_score(model, X_train, y_train, n_jobs=-1, cv=3, scoring='neg_mean_squared_error').mean()
    return score

study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=100)

best_params = study.best_params
best_model.set_params(**best_params)
best_model.fit(X_train, y_train)
y_pred = best_model.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
print(f'Optimized {best_model_name} MSE: {round(mse,2)}')

[I 2024-04-06 12:37:37,796] A new study created in memory with name: no-name-705ccbd9-fa22-453c-a2d6-bf353d34196b
[I 2024-04-06 12:37:40,129] Trial 0 finished with value: -204612060.29724202 and parameters: {'n_estimators': 97, 'max_depth': 27}. Best is trial 0 with value: -204612060.29724202.
[I 2024-04-06 12:37:42,091] Trial 1 finished with value: -210963917.82232925 and parameters: {'n_estimators': 56, 'max_depth': 5}. Best is trial 1 with value: -210963917.82232925.
[I 2024-04-06 12:37:43,803] Trial 2 finished with value: -207104501.47195038 and parameters: {'n_estimators': 35, 'max_depth': 25}. Best is trial 1 with value: -210963917.82232925.
[I 2024-04-06 12:37:43,926] Trial 3 finished with value: -229304955.86360577 and parameters: {'n_estimators': 71, 'max_depth': 2}. Best is trial 3 with value: -229304955.86360577.
[I 2024-04-06 12:37:44,026] Trial 4 finished with value: -233256890.6862544 and parameters: {'n_estimators': 44, 'max_depth': 16}. Best is trial 4 with value: -2332

Optimized Random Forest Regressor MSE: 78102083.88
